<a href="https://colab.research.google.com/github/meisisoiisme/FH-ML_PyTorch/blob/main/Full_pipeline_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Needed Imports

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.transforms.functional import to_pil_image
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
import zipfile

Upload Files

In [53]:
# Upload the synthetic dataset zip file to Colab
uploaded = files.upload()

# Extract the contents of the zip file
dataset_zip_path = next(iter(uploaded))
dataset_root = "./synthetic_dataset"
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_root)


Saving test.zip to test (1).zip


Generate Custom Dataset

In [67]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self.make_dataset()

    def make_dataset(self):
        images = []
        for cls in self.classes:
            class_path = os.path.join(self.root_dir, cls)
            if os.path.isdir(class_path):  # Check if it's a directory
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    if os.path.isfile(img_path):  # Check if it's a file
                        item = (img_path, self.class_to_idx[cls])
                        images.append(item)
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label

Test Dataset

In [46]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = self.make_dataset()

    def make_dataset(self):
        images = []
        for img_name in os.listdir(self.root_dir):
            img_path = os.path.join(self.root_dir, img_name)
            item = (img_path, -1)  # Use -1 as a placeholder label for testing
            images.append(item)
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, _ = self.images[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img

In [57]:
num_classes = 2

class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5),
            nn.Tanh(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(120 * 53 * 53, 84),
            nn.Tanh(),
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super(ResNetModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [68]:
class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=30):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.num_epochs = num_epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train(self):
        self.model.to(self.device)

        for epoch in range(self.num_epochs):
            self.model.train()
            for inputs, labels in self.train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

            self.model.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for inputs, labels in self.val_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)

                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)
                    val_loss += loss.item()

                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print(f'Epoch {epoch+1}/{self.num_epochs}, Loss: {val_loss/len(self.val_loader)}, Validation Accuracy: {correct/total}')

    def grad_cam(self, input_tensor, target_class=None):
        self.model.eval()
        device = next(self.model.parameters()).device

        # Get the output from the final convolutional layer
        final_conv_layer = None
        for layer in self.model.children():
            if isinstance(layer, nn.Conv2d):
                final_conv_layer = layer

        # Get the gradients with respect to the output of the final convolutional layer
        self.model.zero_grad()
        output = self.model(input_tensor.to(device))
        if target_class is None:
            target_class = torch.argmax(output)
        output[:, target_class].backward()

        # Get the feature map from the final convolutional layer
        feature_maps = final_conv_layer.weight.grad
        alpha = torch.mean(feature_maps, dim=(2, 3), keepdim=True)

        # Perform weighted combination to get the heatmap
        heatmap = torch.sum(alpha * feature_maps, dim=1, keepdim=True)
        heatmap = F.relu(heatmap)

        # Normalize the heatmap
        heatmap /= torch.max(heatmap)

        return heatmap

    def overlay_heatmap(self, image, heatmap):
        heatmap = heatmap.squeeze().cpu().numpy()
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        overlayed_image = cv2.addWeighted(image, 0.5, heatmap, 0.5, 0)
        return overlayed_image

    def evaluate_and_save_results(self):
        self.model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        confusion_matrix_result = confusion_matrix(all_labels, all_preds)
        np.savetxt("CM.csv", confusion_matrix_result)
        return confusion_matrix_result


    def visualize_interpretations(self):
        self.model.eval()

        for inputs, _ in self.test_loader:
            inputs = inputs.to(self.device)
            outputs = self.model(inputs)

            # Visualize Grad-CAM for each class
            for class_idx in range(outputs.size(1)):
                heatmap = self.grad_cam(inputs, target_class=class_idx)
                original_image = self.convert_tensor_to_image(inputs)
                overlaid_image = self.overlay_heatmap(original_image, heatmap)

                # Display images using Matplotlib
                fig, axes = plt.subplots(1, 2, figsize=(10, 5))
                axes[0].imshow(np.transpose(original_image, (1, 2, 0)))
                axes[0].set_title('Original Image')

                axes[1].imshow(overlaid_image)
                axes[1].set_title('Overlaid Image with Grad-CAM')

                plt.show()

                # Save images using OpenCV
                cv2.imwrite(f'original_image_class_{class_idx}.jpg', cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR))
                cv2.imwrite(f'overlaid_image_class_{class_idx}.jpg', cv2.cvtColor(overlaid_image, cv2.COLOR_RGB2BGR))

                # Break after processing the first class for brevity
                break
            break  # Break after processing the first batch for brevity

    def convert_tensor_to_image(self, tensor):
        # Ensure the tensor is on CPU and in the range [0, 1]
        tensor = tensor.cpu().clamp(0, 1)

        # Convert the tensor to a PIL Image
        image = to_pil_image(tensor)

        return image





Main

1) Load Datasets
2) Build Models
3) Train Models
4) Evaluate Models
5) Visualize Results  
6) Save Models

In [65]:
if __name__ == "__main__":
    # Data transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load datasets
    train_dataset = CustomDataset(root_dir=dataset_root, transform=transform)
    test_dataset = TestDataset(root_dir=dataset_root, transform=transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Build models
    alexnet_model = AlexNet(num_classes=len(train_dataset.classes))
    lenet5_model = LeNet5(num_classes=len(train_dataset.classes))
    resnet_model = ResNetModel(num_classes=len(train_dataset.classes))

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer_alexnet = optim.RMSprop(alexnet_model.parameters(), lr=1e-4)
    optimizer_lenet5 = optim.RMSprop(lenet5_model.parameters(), lr=1e-4)
    optimizer_resnet = optim.RMSprop(resnet_model.parameters(), lr=1e-4)

    # AlexNet - Train, save, evaluate, visualize
    trainer_alexnet = ModelTrainer(alexnet_model, train_loader, test_loader, test_loader, criterion, optimizer_alexnet)
    trainer_alexnet.train()
    trainer_alexnet.evaluate_and_save_results()
    trainer_alexnet.visualize_interpretations()

    # LeNet5 - Train, save, evaluate, visualize
    trainer_lenet5 = ModelTrainer(lenet5_model, train_loader, test_loader, test_loader, criterion, optimizer_lenet5)
    trainer_lenet5.train()
    trainer_lenet5.evaluate_and_save_results()
    trainer_lenet5.visualize_interpretations()

    # ResNet - Train, save, evaluate, visualize
    trainer_resnet = ModelTrainer(resnet_model, train_loader, test_loader, test_loader, criterion, optimizer_resnet)
    trainer_resnet.train()
    trainer_resnet.evaluate_and_save_results()
    trainer_resnet.visualize_interpretations()

IsADirectoryError: [Errno 21] Is a directory: './synthetic_dataset/class3'